In [6]:
import socket
import torch
import pickle
from transformers import AutoModel

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Accept connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Load TinyBERT part 2
tinybert = AutoModel.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

class TinyBERTPart2(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart2, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.encoder.layer[2:])
        self.pooler = original_model.pooler

    def forward(self, x):
        for layer in self.encoder_layers:
            x = layer(x)[0]  # First element of the output tuple
        x = self.pooler(x)
        return x

model_part2 = TinyBERTPart2(tinybert)

# Perform forward pass through part 2
with torch.no_grad():
    output = model_part2(intermediate_output)
print(f"Model Part 2 forward pass completed. Output shape: {output.shape}")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.19.49', 63432)
Intermediate output received from client.
Deserialized the received data.
Model Part 2 forward pass completed. Output shape: torch.Size([1, 312])
Connection closed.


In [ ]:
import socket

# Create a socket object
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Define the host and port
host = '0.0.0.0'  # This will allow connections from any network interface
port = 10300       # Choose a port number that is not being used

# Bind the socket to the host and port
server_socket.bind((host, port))

# Enable the server to listen for incoming connections (max 1 connection)
server_socket.listen(1)
print(f"Server is listening on {host}:{port}...")

# Accept a connection
client_socket, client_address = server_socket.accept()
print(f"Connected to {client_address}")

# Communication loop
while True:
    # Receive data from the client (max 1024 bytes)
    data = client_socket.recv(1024).decode()
    if not data:
        break
    print(f"Client: {data}")

    # Send a reply to the client
    message = input("Server (you): ")
    client_socket.send(message.encode())

# Close the connection
client_socket.close()
server_socket.close()

Server is listening on 0.0.0.0:10300...
Connected to ('172.16.19.49', 64415)
Client: hi


Server (you):  Hello Client


Client: careless you are


Server (you):  Thank you


In [ ]:
import socket
import torch
import pickle
from transformers import AutoModel
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Function to calculate the total number of parameters in a model
def get_model_size(model):
    return sum(p.numel() for p in model.parameters())

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load the original TinyBERT model
tinybert = AutoModel.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

# Create the split models
class TinyBERTPart1(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart1, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.encoder.layer[:2])  # First 2 layers

    def forward(self, x):
        for layer in self.encoder_layers:
            x = layer(x)[0]  # First element of the output tuple
        return x

class TinyBERTPart2(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart2, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.encoder.layer[2:])  # Remaining layers
        self.pooler = original_model.pooler
        self.classifier = nn.Linear(original_model.config.hidden_size, 2)  # Binary classification

    def forward(self, x):
        for layer in self.encoder_layers:
            x = layer(x)[0]
        x = self.pooler(x)
        x = self.classifier(x)
        return x

model_part1 = TinyBERTPart1(tinybert)
model_part2 = TinyBERTPart2(tinybert)

# Accept connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Perform forward pass through Part 2
with torch.no_grad():
    output = model_part2(intermediate_output)

print(f"Model Part 2 forward pass completed. Output shape: {output.shape}")

# Step 1: Convert the output to logits
predictions = torch.argmax(output, dim=1)  # Get predicted class indices

# Step 2: For evaluation, assume you have a list of true labels
# Replace this with your actual true labels for the evaluation
true_labels = torch.tensor([1, 0, 1, 1, 0])  # Example true labels

# Calculate accuracy
accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


Server listening on 0.0.0.0:10300...


In [1]:
import socket
import torch
import pickle
from transformers import AutoModel
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load the original TinyBERT model
tinybert = AutoModel.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

class TinyBERTPart1(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart1, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.encoder.layer[:2])  # First 2 layers

    def forward(self, x):
        for layer in self.encoder_layers:
            x = layer(x)[0]  # Process through each layer
        return x

class TinyBERTPart2(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart2, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.encoder.layer[2:])  # Remaining layers
        self.pooler = original_model.pooler
        self.classifier = nn.Linear(original_model.config.hidden_size, 2)  # Binary classification

    def forward(self, x):
        for layer in self.encoder_layers:
            x = layer(x)[0]  # Process through each layer
        x = self.pooler(x)  # Apply the pooler
        x = self.classifier(x)  # Classification layer
        return x

model_part1 = TinyBERTPart1(tinybert)
model_part2 = TinyBERTPart2(tinybert)

# Accept connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Perform forward pass through Part 2
with torch.no_grad():
    output = model_part2(intermediate_output)

print(f"Model Part 2 forward pass completed. Output shape: {output.shape}")

# Step 1: Convert the output to logits
predictions = torch.argmax(output, dim=1)  # Get predicted class indices

# Step 2: For evaluation, assume you have a list of true labels
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

# Calculate accuracy
accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


C:\Users\Exam\.conda\envs\Rakuten\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.19.49', 64244)
Intermediate output received from client.
Deserialized the received data.
Model Part 2 forward pass completed. Output shape: torch.Size([5, 2])
Accuracy: 60.00%
Connection closed.


In [2]:
predictions

tensor([1, 1, 0, 0, 1])

In [3]:
true_labels

tensor([1, 0, 1, 0, 1])

In [4]:
model_part2

TinyBERTPart2(
  (encoder_layers): ModuleList(
    (0-1): 2 x BertLayer(
      (attention): BertAttention(
        (self): BertSdpaSelfAttention(
          (query): Linear(in_features=312, out_features=312, bias=True)
          (key): Linear(in_features=312, out_features=312, bias=True)
          (value): Linear(in_features=312, out_features=312, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=312, out_features=312, bias=True)
          (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=312, out_features=1200, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=1200, out_features=312, bias=True)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise